##Subset VLA
This routine filters out all but every *Nfrac*-th line of each segment of the .vla file. Note right now the code isn't smart enough to make sure that it includes the endpoint of each segment - should probably fix that.

In [1]:
    Nfrac = 40
    readfile=open('Shows\models\SDvision_Laniakea_streamlines_v001.vla',"r")
    writefile=open('Shows\models\Laniakea_streamlines_p025.vla',"w")
    count = 1
    for line in readfile:
        if (line.find('P ')==0):
            if (count !=1):
                writefile.write(oldline)                
                count=1;
        if (line.find('L ')!=0 or count%Nfrac ==0):
            writefile.write(line)
        if (line.find('L ')==0):
            count=count+1
            oldline=line
    writefile.close()
    readfile.close()

##Convert VLA
This routine converts the km/s units to Mpc in a .vla file.


In [2]:
from astropy import units as u
from astropy.cosmology import Planck13
readfile=open('Shows\models\Laniakea_streamlines_p025.vla',"r")
writefile=open('Shows\models\Laniakea_streamlines_Mpc_p025.vla',"w")
for line in readfile:
    if (line.find('L ')!=0 and line.find('P ')!=0):
        #pass along the header and blank lines
        writefile.write(line)
    else:
        elements = line.split()
        outline = elements[0]+ " %7.3f  %7.3f  %7.3f 1\n" % \
          (((float(elements[1])*u.km/u.s)/Planck13.H(0)).value, \
          ((float(elements[2])*u.km/u.s)/Planck13.H(0)).value, \
          ((float(elements[3])*u.km/u.s)/Planck13.H(0)).value )
        writefile.write(outline)
writefile.close()
readfile.close()

In [39]:
Planck13.H(0)

<Quantity 67.77 km / (Mpc s)>